In [1]:
import sys
import os
import yaml

sys.path.append(os.getenv("CODE_PATH"))
sys.path.append(os.getenv("FIN_DATABASE_PATH"))


import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import logging
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import ta
from log_config import setup_logging
from Data.connect import engine, DailyStockData, HourlyStockData, OneMinuteStockData, FiveMinuteStockData,FifteenMinuteStockData, StockSplits, StockNews, CompanyFinancials
from Pre_Processing.pre_processing import PreProcessing
from Feature_Engineering.feature_engineering import TechnicalIndicators
from pipeline import Pipeline

from sklearn.metrics import roc_auc_score, roc_curve, auc

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, log_loss, confusion_matrix, ConfusionMatrixDisplay

In [2]:
wiki = 'http://en.wikipedia.org/wiki'
djia_ticker_list = wiki + '/Dow_Jones_Industrial_Average'
sp500_tickers_list = wiki + '/List_of_S%26P_500_companies'
tickersSP500 = pd.read_html(sp500_tickers_list)[0].Symbol.to_list()
djia_tickers = pd.read_html(djia_ticker_list)[1].Symbol.to_list()

In [3]:
pipe = Pipeline(djia_tickers)

In [4]:
a = pipe.pipeline()['AAPL']

In [10]:
data = pipe.pipeline()

In [11]:
data

{'MMM':                 id       date      timestamp ticker      open      high  \
 date                                                                      
 2003-09-10   35531 2003-09-10  1063166400000    MMM   57.4457   57.6505   
 2003-09-11  190611 2003-09-11  1063252800000    MMM   57.1363   58.3738   
 2003-09-12  190612 2003-09-12  1063339200000    MMM   57.8387   58.2107   
 2003-09-15  190613 2003-09-15  1063598400000    MMM   57.9223   58.1564   
 2003-09-16  190614 2003-09-16  1063684800000    MMM   57.9766   58.8629   
 ...            ...        ...            ...    ...       ...       ...   
 2024-08-05  195870 2024-08-05  1722830400000    MMM  119.5700  125.3154   
 2024-08-06  195871 2024-08-06  1722916800000    MMM  123.3600  126.1300   
 2024-08-07  195872 2024-08-07  1723003200000    MMM  125.3900  126.5700   
 2024-08-08  195873 2024-08-08  1723089600000    MMM  122.4500  125.7400   
 2024-08-09  195874 2024-08-09  1723176000000    MMM  124.6400  124.6400   
 
   

In [13]:
combined_df = pd.concat(
    [df.assign(ticker=ticker) for ticker, df in data.items()],
    axis=0
)

# Optionally, you can reset the index if needed
combined_df.reset_index(drop=True, inplace=True)

# Set the index names as 'date' and 'ticker'
combined_df.set_index(['date', 'ticker'], inplace=True)

# Lowercase all column names
combined_df.columns = combined_df.columns.str.lower()

# Display the combined DataFrame
combined_df


,,id,timestamp,open,high,low,close,volume,vwap,transactions,rsi_14,...,ema_50,sma_100,ema_100,sma_200,ema_200,sma_300,ema_300,day,month,day_of_week
date,ticker,,,,,,,,,,,,,,,,,,,,,
2003-09-10,MMM,35531,1063166400000,57.4457,57.6505,56.9858,57.1070,4675642.4,57.2454,3656,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,9,2
2003-09-11,MMM,190611,1063252800000,57.1363,58.3738,57.1363,58.0644,5067452.0,58.0316,3995,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,9,3
2003-09-12,MMM,190612,1063339200000,57.8387,58.2107,57.5878,57.7885,5220061.6,57.8624,3930,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,9,4
2003-09-15,MMM,190613,1063598400000,57.9223,58.1564,57.4666,57.8721,4280962.4,57.8385,3678,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,9,0
2003-09-16,MMM,190614,1063684800000,57.9766,58.8629,57.9641,58.8629,3885804.0,58.4185,3353,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-05,WMT,346684,1722830400000,68.1000,68.5800,66.9100,67.5900,20532419.0,67.4661,195503,40.339044,...,67.745380,64.3732,64.962920,59.472500,61.113797,57.179824,58.595099,5,8,0
2024-08-06,WMT,346685,1722916800000,67.8400,68.6300,67.6800,67.7400,14511713.0,68.1010,123990,41.529132,...,67.745169,64.4378,65.017912,59.541967,61.179729,57.243224,58.655862,6,8,1
2024-08-07,WMT,346686,1723003200000,67.8700,68.0100,66.8100,66.9100,17910124.0,67.2792,143908,37.117138,...,67.712417,64.4967,65.055379,59.608567,61.236747,57.303569,58.710707,7,8,2


In [15]:
combined_df = pd.concat(
    [df.assign(ticker=ticker).set_index('ticker', append=True) for ticker, df in data.items()]
)

combined_df = combined_df.reorder_levels(['date', 'ticker'])
combined_df.columns = combined_df.columns.str.lower()
combined_df.sort_index(inplace=True)
combined_df


id       date      timestamp      open      high  \
date       ticker                                                         
2003-09-10 AAPL    211670 2003-09-10  1063166400000    0.3973    0.4038   
           AMGN    201140 2003-09-10  1063166400000   67.2500   68.1400   
           AMZN    206405 2003-09-10  1063166400000    2.3130    2.3615   
           AXP     195875 2003-09-10  1063166400000   45.5200   45.6900   
           BA      216935 2003-09-10  1063166400000   36.1800   36.4900   
...                   ...        ...            ...       ...       ...   
2024-08-09 TRV     326035 2024-08-09  1723176000000  211.9900  212.6100   
           UNH     331300 2024-08-09  1723176000000  564.5400  565.9300   
           V       341423 2024-08-09  1723176000000  259.1700  261.5200   
           VZ      336565 2024-08-09  1723176000000   40.4900   40.8000   
           WMT     346688 2024-08-09  1723176000000   67.8500   68.2200   

                        low     close       volume      vwap  transactions  \
date       ticker                                                            
2003-09-10 AAPL      0.3948    0.3961  224893256.0    0.3988         10152   
           AMGN     66.8800   66.9900    7302682.0   67.4818         21885   
           AMZN      2.3030    2.3110  160193680.0    2.3317         24593   
           AXP      44.9000   45.0500    3619400.0   45.3662          4472   
           BA       35.3300   35.6200    4058000.0   35.8444          4776   
...                     ...       ...          ...       ...           ...   
2024-08-09 TRV     209.8900  212.4800     963916.0  211.9353         18284   
           UNH     557.6300  558.7600    2864613.0  560.3721         70237   
           V       259.0100  259.7600    7154274.0  260.1813         93436   
           VZ       40.0300   40.7500    9529207.0   40.6122         73311   
           WMT      67.3300   67.9500   12959879.0   67.8626        109903   

                   ...      ema_50   sma_100     ema_100     sma_200  \
date       ticker  ...                                                 
2003-09-10 AAPL    ...         NaN       NaN         NaN         NaN   
           AMGN    ...         NaN       NaN         NaN         NaN   
           AMZN    ...         NaN       NaN         NaN         NaN   
           AXP     ...         NaN       NaN         NaN         NaN   
           BA      ...         NaN       NaN         NaN         NaN   
...                ...         ...       ...         ...         ...   
2024-08-09 TRV     ...  211.263416  214.4693  210.631456  204.819150   
           UNH     ...  534.082387  505.6354  519.639264  514.166100   
           V       ...  266.339551  271.9036  268.427221  267.656650   
           VZ      ...   40.523580   40.4687   40.261731   39.538050   
           WMT     ...   67.719759   64.6374   65.163253   59.753667   

                      ema_200     sma_300     ema_300  day  month  day_of_week  
date       ticker                                                               
2003-09-10 AAPL           NaN         NaN         NaN   10      9            2  
           AMGN           NaN         NaN         NaN   10      9            2  
           AMZN           NaN         NaN         NaN   10      9            2  
           AXP            NaN         NaN         NaN   10      9            2  
           BA             NaN         NaN         NaN   10      9            2  
...                       ...         ...         ...  ...    ...          ...  
2024-08-09 TRV     204.499439  192.523800  198.606524    9      8            4  
           UNH     512.253386  507.843500  508.591919    9      8            4  
           V       264.259281  257.361667  257.979902    9      8            4  
           VZ       39.539018   37.636133   39.430052    9      8            4  
           WMT      61.366822   57.430124   58.831166    9      8            4  

[156079 rows x 42 columns]

In [ ]:
prices.groupby(level='ticker').close.apply(RSI)

In [ ]:
data = pipe.pipeline(timespan = 'hour')

In [ ]:
apple_data = data['AAPL']

In [ ]:
fig = go.Figure(data = go.Candlestick(x = apple_data.index,
                                      open = apple_data['open'],
                                      high = apple_data['high'],
                                      low = apple_data['low'],
                                      close = apple_data['close'],
                                      increasing=dict(line=dict(color='black')),
                                      decreasing=dict(line=dict(color='red')),
                                      showlegend=False))

layout = go.Layout(
    title=' Apple Stock Price - 2003 to 2024',
    yaxis=dict(title='Price (USD)'),
    xaxis=dict(title='Date'),
    template = 'ggplot2',
    xaxis_rangeslider_visible=False,
    yaxis_gridcolor='white',
    xaxis_gridcolor='white',
    yaxis_tickfont=dict(color='black'),
    xaxis_tickfont=dict(color='black'),
    margin=dict(t=50,l=50,r=50,b=50)
)

fig.update_layout(layout)


fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = apple_data.index,
                         y = apple_data['return'].dropna(),
                         mode = 'lines',
                         line = dict(color = 'blue'),
                         name = 'Returns'))


layout = go.Layout(
    title=' Apple Stock Returns - 2003 to 2024',
    yaxis=dict(title='Returns (%)'),
    xaxis=dict(title='Date'),
    template='ggplot2',
    xaxis_rangeslider_visible=False,
    yaxis_gridcolor='white',
    xaxis_gridcolor='white',
    yaxis_tickfont=dict(color='black'),
    xaxis_tickfont=dict(color='black'),
    margin=dict(t=50, l=50, r=50, b=50)
)

fig.update_layout(layout)

fig.show()

In [ ]:
import plotly.figure_factory as ff

# Plot the distribution (Histogram + KDE)
fig = ff.create_distplot([apple_data['return'].dropna()], ['Returns'], bin_size=0.001, show_rug=False)

# Updating the layout
fig.update_layout(
    title='Distribution of Apple Stock Returns',
    xaxis_title='Returns',
    yaxis_title='Density',
    template='ggplot2',
    margin=dict(t=50, l=50, r=50, b=50)
)

fig.show()


In [ ]:
import scipy.stats as stats

# QQ plot
plt.figure(figsize=(8, 6))
stats.probplot(apple_data['return'].dropna(), dist="norm", plot=plt)
plt.title('QQ Plot of Apple Stock Returns')
plt.show()


In [ ]:
apple_data['target'] = np.where(apple_data['log_ret'] > 0,1,
                                0)



In [ ]:
apple_data['target'] = apple_data['target'].shift(-1)

In [ ]:
apple_data = apple_data.dropna()

In [ ]:
features = [ 'date', 'timestamp', 'open', 'high', 'low', 'close',
       'volume', 'vwap', 'transactions', 'RSI_14', 'RSI_2', 'MACD', 'log_ret',
       'ROC', 'Stoch', 'ADX', 'ADX_pos', 'ADX_neg', 'rolling_H-L_25',
       'lower_band', 'ATR', 'IBS', 'OBV', 'ema_5', 'ema_10',
        'ema_30', 'ema_50', 'ema_100',
       'ema_200', 'ema_300']

In [ ]:
#Normal CV 
train_test_split = 0.7
train_idx = int(len(apple_data) * train_test_split)

train_data = apple_data.iloc[:train_idx]
test_data = apple_data.iloc[train_idx:]

In [ ]:
X_train = train_data.drop(columns = ['id', 'date', 'timestamp',
                          'ticker', 'target', 'return'])

y_train = train_data['target']
X_test = test_data.drop(columns = ['id', 'date', 'timestamp',
                          'ticker', 'target', 'return'])
y_test = test_data['target']

In [ ]:
xgboost = XGBClassifier()

In [ ]:
xgboost.fit(X_train,y_train)


In [ ]:
y_pred = xgboost.predict_proba(X_test)[:, 1]

In [ ]:
roc_auc = roc_auc_score(y_test, y_pred)
log_loss_value = log_loss(y_test, y_pred)

In [ ]:
log_loss_value

In [ ]:
roc_auc

In [ ]:
plt.figure(figsize=(10, 10))
for name, model in models:
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1] 
    y_pred = model.predict(X_test) 
    
    # Metrics
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    log_loss_value = log_loss(y_test, y_pred_proba)

    print(f'{name} ROC AUC: {roc_auc}')
    print(f'{name} Log Loss: {log_loss_value}')
    
    # ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    
    #CM
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f'{name} Confusion Matrix')
    plt.show()

In [ ]:
web.DataReader('F-F_Research_Data_5_Factors_2x3',
                               'famafrench',
                               start='2010')[0].drop('RF', axis=1)

In [ ]:
import pandas_datareader.data as web


In [ ]:
models = [
    ("XGBoost", XGBClassifier()),
    ("CatBoost", CatBoostClassifier(verbose=False)),
    ("RandomForest", RandomForestClassifier())
]

In [ ]:
plt.figure(figsize=(10, 10))
for name, model in models:
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1] 
    y_pred = model.predict(X_test) 
    
    # Metrics
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    log_loss_value = log_loss(y_test, y_pred_proba)

    print(f'{name} ROC AUC: {roc_auc}')
    print(f'{name} Log Loss: {log_loss_value}')
    
    # ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    
    #CM
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f'{name} Confusion Matrix')
    plt.show()